# 上週作業

## 這個部份上周剛好講過，所以跳過，自己看。

In [ ]:
import os

os.chdir("../../../")

In [ ]:
import json

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

from src.io.path_definition import get_project_dir


embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Ingredients: {input}\nOrigin: {output}",
)

with open(os.path.join(get_project_dir(), 'tutorial', 'LLM+Langchain', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

examples = []

for recipe in recipe_train[:1000]:
    examples.append({"input": " ".join(recipe['ingredients']),
                     "output": recipe['cuisine']})

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=5,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Find the recipe origin based on the ingredients",
    suffix="Ingredients: {ingredients}\nOrigin:",
    input_variables=["ingredients"],
)

In [ ]:
similar_prompt

In [ ]:
with open(os.path.join(get_project_dir(), 'tutorial', 'LLM+Langchain', 'Week-1', 'recipe_test.json'), 'r') as f:
    recipe_test = json.load(f)

existing_ingredients = recipe_test[99]['ingredients']

similar_prompt.invoke(", ".join(existing_ingredients))

In [ ]:
from langchain.chat_models import ChatOpenAI

from src.initialization import credential_init

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

In [ ]:
pipeline_ = similar_prompt|model

In [ ]:
pipeline_.invoke(", ".join(existing_ingredients))

##  飛安報告

In [ ]:
from src.io.path_definition import get_project_dir

import pandas as pd

df = pd.read_excel(os.path.join(get_project_dir(), 'tutorial', 'LLM+Langchain', 'Week-3', 'Data sample.xlsx'))

### 回家作業 1

若要飛安事故報告可以有複數分類結果，如何調整Prompt，包含parser

In [ ]:
def build_standard_chat_prompt_template_v2(kwargs):

    messages = []
    
    for key in ['system', 'few_shot', 'human']:
        if kwargs.get(key):
            if key == 'system':
                system_content = kwargs['system']
                system_prompt = PromptTemplate(**system_content)
                message = SystemMessagePromptTemplate(prompt=system_prompt)
            elif key == 'human':
                human_content = kwargs['human']
                human_prompt = PromptTemplate(**human_content)
                message = HumanMessagePromptTemplate(prompt=human_prompt)
            else:
                few_shot_content = kwargs['few_shot']
                message = FewShotChatMessagePromptTemplate(**few_shot_content)
            
            messages.append(message)
    
    chat_prompt = ChatPromptTemplate.from_messages(messages)

    return chat_prompt

In [ ]:
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


system_template = '''You are an AI assistant assigned with a task of safty report classification based on the content.
    You are a seasoned flight safety inspector with deep and extensive knowledge of aviation safty. 
    
    You always do the best work you can. You are highly analytical and pay close attention to details. 
    
    The candidates of the output are:

    - `Organizational Influence;Resource Management`
    - `Organizational Influence;Organizational Climate`
    - `Organizational Influence;rganizational Process`
    - `Unsafe Supervisions;Inadequate Supervision`
    - `Unsafe Supervisions;Planned Inappropriate Operations`
    - `Unsafe Supervisions;Failed to Correct Problem`
    - `Unsafe Supervisions;Supervisory Violation`
    - `Precondition for Unsafe Acts;Environmental Factors;Physical Environment`
    - `Precondition for Unsafe Acts;Environmental Factors;Technological Environment`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Physiological State`
    - `Precondition for Unsafe Acts;Condition of Operators;Physical/Mental Limitations`
    - `Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management`
    - `Precondition for Unsafe Acts;Personnel Factors;Personal Readiness`
    - `Unsafe Acts;Errors;Decision Errors`
    - `Unsafe Acts;Errors;Skill-Based Errors`
    - `Unsafe Acts;Errors;Perceptual Errors`
    - `Unsafe Acts;Violations;Routine`
    - `Unsafe Acts;Violations;Exceptional`

     The report can involve multiple categories.
    '''


human_template = """{report}; format instruction: {format_instructions}"""

response_schemas = [
        ResponseSchema(name="categories", description="The predicted categories as a python list")
    ]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["report"],
                    "partial_variables": {'format_instructions': format_instructions}}}

chat_prompt_template = build_standard_chat_prompt_template_v2(input_)

pipeline_ = chat_prompt_template|model|output_parser

In [ ]:
text = df.iloc[3]['Report 1']
print(text)

In [ ]:
output = pipeline_.invoke({"report": text})
print(output)

### 回家作業 2

你可以很清楚的看到一個飛安事故中，可以出現複數報告。
將`Report 1` 和 `Report 1.2` 結合起來產生一份的新報告。

抄也是一門技術

In [ ]:
system_template = """You are an AI assistant assigned with a task of safty report classification based on the content.
    You are a seasoned flight safety inspector with deep and extensive knowledge of aviation safty. 

    You will receieve two reports <report_1> and <report_2> and you will consolidate the content before drawing conclusion. 
    
    You always do the best work you can. You are highly analytical and pay close attention to details. 
    
    The candidates of the output are:

    - `Organizational Influence;Resource Management`
    - `Organizational Influence;Organizational Climate`
    - `Organizational Influence;rganizational Process`
    - `Unsafe Supervisions;Inadequate Supervision`
    - `Unsafe Supervisions;Planned Inappropriate Operations`
    - `Unsafe Supervisions;Failed to Correct Problem`
    - `Unsafe Supervisions;Supervisory Violation`
    - `Precondition for Unsafe Acts;Environmental Factors;Physical Environment`
    - `Precondition for Unsafe Acts;Environmental Factors;Technological Environment`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Mental State`
    - `Precondition for Unsafe Acts;Condition of Operators;Adverse Physiological State`
    - `Precondition for Unsafe Acts;Condition of Operators;Physical/Mental Limitations`
    - `Precondition for Unsafe Acts;Personnel Factors;Crew Resource Management`
    - `Precondition for Unsafe Acts;Personnel Factors;Personal Readiness`
    - `Unsafe Acts;Errors;Decision Errors`
    - `Unsafe Acts;Errors;Skill-Based Errors`
    - `Unsafe Acts;Errors;Perceptual Errors`
    - `Unsafe Acts;Violations;Routine`
    - `Unsafe Acts;Violations;Exceptional`

     The report can involve multiple categories.
    """

human_template = """
                 report_1:{report_1}; 
                 report_2: {report_2};
                 format instruction: {format_instructions}
                 """

response_schemas = [
        ResponseSchema(name="categories", description="The predicted categories as a python list")
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["report_1, report_2"],
                    "partial_variables": {'format_instructions': format_instructions}}}

chat_prompt_template = build_standard_chat_prompt_template_v2(input_)

pipeline_ = chat_prompt_template|model|output_parser

In [ ]:
df.iloc[3]['Report 1.2']

In [ ]:
output = pipeline_.invoke({"report_1": text,
                       "report_2": df.iloc[3]['Report 1.2']})

print(output['categories'])

### Keynote

- 若你更理解你的數據，你可以建立更精確的Prompt，更明確的表示每個數據代表的意義，來提升輸出的品質。

# Remote server

### 1. Making a POST Request (發送 POST 請求):

- requests.post(...) sends an HTTP POST request to the specified URL.
- The URL "http://localhost:5000/openai/invoke" points to a local server running on port 5000, at the endpoint /openai/invoke.
- The json parameter is used to send a JSON payload with the request. In this case, the payload is {'input': "Where is Taiwan"}.
- requests.post(...) 發送一個 HTTP POST 請求到指定的 URL。
- URL "http://localhost:5000/openai/invoke" 指向一個本地服務器，該服務器在端口 5000 上運行，並且指向 /openai/invoke 端點。
- json 參數用於隨請求發送 JSON 負載。在這個例子中，負載是 {'input': "Where is Taiwan"}。

### 2. Response Handling (響應處理):

- The server processes the request and sends back a response.
- The response is stored in the response variable, which can then be inspected or used further in the code.
- 服務器處理請求並返回響應。
- 響應存儲在 response 變量中，之後可以檢查或在代碼中進一步使用。

In [ ]:
import requests

response = requests.post(
    "http://localhost:5000/openai/invoke",
    json={'input': "Where is Taiwan"}
)

In [ ]:
response.json()

In [ ]:
response.json()['output']['content']

# Use the remote model as `Software as a service` (SaaS)

## Basic Usage

### 1. Creating an Instance of RemoteRunnable (創建 RemoteRunnable 的實例):

- This line creates an instance of RemoteRunnable and initializes it with the URL of the remote language model service. In this case, the service is running locally on http://localhost:5000/openai/.
- 這行代碼創建一個 RemoteRunnable 的實例，並用遠程語言模型服務的 URL 進行初始化。在這個例子中，服務在本地運行，URL 為 http://localhost:5000/openai/。

In [ ]:
from langserve import RemoteRunnable

llm = RemoteRunnable("http://localhost:5000/openai/")

### 2. Asynchronous Streaming of Responses (異步流式處理回應):

- llm.astream("Where is Taiwan?") sends the query "Where is Taiwan?" to the remote service and retrieves the response as a stream.
- async for msg in ... is used to handle the streaming responses asynchronously.
- print(msg.content, end="", flush=True) prints each message content received from the stream without adding a new line after each message, and flushes the output buffer to ensure the message is displayed immediately.
- llm.astream("Where is Taiwan?") 將查詢 "Where is Taiwan?" 發送到遠程服務，並以流的形式檢索回應。
- async for msg in ... 用於異步處理流式回應。
- print(msg.content, end="", flush=True) 打印每個從流中接收到的消息內容，不在每個消息後添加新行，並刷新輸出緩衝區以確保消息立即顯示。

In [ ]:
# Supports astream
async for msg in llm.astream("Where is Taiwan?"):
    print(msg.content, end="", flush=True)

In [ ]:
output = llm.invoke("Where is Taiwan?")

In [ ]:
output.content

## Make the external service a part of the chain

### 1. Comedian Chain (喜劇演員鏈)

- ChatPromptTemplate.from_messages(...) creates a prompt template where the system prompt instructs the model to either tell a joke or state a fact, and the human prompt provides the input.
- This template is then piped (|) to a language model (llm) to generate the comedian's response.
- ChatPromptTemplate.from_messages(...) 創建一個提示模板，其中系統提示指示模型要麼講一個笑話，要麼陳述一個不搞笑的事實，並且僅輸出一個。
- 然後將此模板通過管道（|）傳遞給語言模型（llm），以生成喜劇演員的回應。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

system_template = """
                  You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. 
                  Please either tell a joke or state fact now but only output one.
                  """

human_template = """
                 {input}
                 """


input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["input"]}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

comedian_pipeline_ = chat_prompt_template|llm


### 2. Joke Classifier Chain

- This chain is similar to the comedian chain but serves a different purpose.
- The system prompt asks the model to classify the joke as "funny" or "not funny" and repeat the first five words for reference.
- This template is also piped to the language model (llm).
- 這個鏈與喜劇演員鏈類似，但用途不同。
- 系統提示要求模型將笑話分類為“搞笑”或“不搞笑”，並重複笑話的前五個詞以供參考。
- 此模板也通過管道傳遞給語言模型（llm）。

In [ ]:
system_template = """
                  Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. 
                  Then repeat the first five words of the joke for reference...
                  """

human_template = """
                 {input}
                 """

input_ = {"system": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["input"]}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

joke_classifier_pipeline_ = chat_prompt_template|llm


### 3. Combining Chains with RunnablePassthrough

- This combines the comedian chain and the joke classifier chain using RunnablePassthrough.assign.
- The comedian chain generates the output, and then this output is passed to the joke classifier chain to classify its humor.
- 這將喜劇演員鏈和笑話分類器鏈結合在一起，使用 RunnablePassthrough.assign。
- 喜劇演員鏈生成輸出，然後將此輸出傳遞給笑話分類器鏈以分類其幽默性。

In [ ]:
pipeline_ = {"joke": comedian_chain} | RunnablePassthrough.assign(
    classification=joke_classifier_chain
)

In [ ]:
pipeline_.invoke({"input": "A man and a beer"})

# ChatBot

In [ ]:
from IPython.display import Image

Image(url="https://python.langchain.com/v0.1/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png")

- N-Shot
- The historical chat history can be consdiered as a list of question-answer pairs
- If the chatbot doesn’t remember past chats, it’s called stateless because it doesn’t know what happened before.

## Minimal Example

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

message = HumanMessage(
            content="Translate this sentence from English to Chinese (繁體中文): I love programming."
        )

model.invoke(
    [message]
)

In [ ]:
model.invoke([HumanMessage(content="What did you just say?")])

The `memory` is not there, so it does not understand your question.

The following example shows how to add memory into the conversation.

In [ ]:
model.invoke(
    [
        HumanMessage(
            content="Translate this sentence from English to  Chinese (繁體中文): I love programming."
        ),
        AIMessage(content="我愛程式設計."),
        HumanMessage(content="What did you just say?"),
    ]
)

## Prompt templates

### 1. Creating a ChatPromptTemplate 

- ChatPromptTemplate.from_messages(...) creates a prompt template for the chatbot.
- The first message in the template is a system message: "You are a helpful assistant. Answer all questions to the best of your ability." This message sets the context and behavior of the assistant, instructing it to be helpful and thorough in its responses.
- MessagesPlaceholder(variable_name="messages") is a placeholder for dynamic content. The variable_name="messages" specifies that this placeholder will be filled with user messages during the conversation.
- ChatPromptTemplate.from_messages(...) 創建了一個聊天機器人的提示模板。
- 模板中的第一條消息是一條系統消息：“You are a helpful assistant. Answer all questions to the best of your ability.” 此消息設置了助手的上下文和行為，指示其在回答中要提供幫助並盡力而為。
- MessagesPlaceholder(variable_name="messages") 是一個動態內容的佔位符。variable_name="messages" 指定該佔位符將在對話中插入用戶消息。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = PromptTemplate(template="""
                                        You are a helpful assistant. Answer all questions to the best of your ability.
                                        """)
system_message = SystemMessagePromptTemplate(prompt=system_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="messages"),
    ]
)

### 2. Creating the Chain

- This line pipes (|) the prompt template to a language model (model).
- chain represents a sequence of operations where the prompt template is used to format user messages, and the language model processes these messages to generate responses.
- 這行代碼通過管道（|）將提示模板傳遞給語言模型（model）。
- chain 代表一系列操作，其中提示模板用於格式化用戶消息，語言模型處理這些消息以生成回應。

In [ ]:
pipeline_ = prompt | model

The MessagesPlaceholder above inserts chat messages passed into the chain's input as chat_history directly into the prompt. Then, we can invoke the chain like this:

In [ ]:
pipeline_.invoke(
    {
        "messages": [
            HumanMessage(
            content="Translate this sentence from English to  Chinese (繁體中文): I love programming."
            ),
            AIMessage(content="我愛程式設計."),
            HumanMessage(content="What did you just say?"),
            ],
    }
)

## Example of Using MessageHistory

As a shortcut for managing the chat history, we can use a MessageHistory class, which is responsible for saving and loading chat messages. There are many built-in message history integrations that persist messages to a variety of databases, but for this quickstart we'll use a in-memory, demo message history called ChatMessageHistory

### 1. Importing the ChatMessageHistory Class (導入 ChatMessageHistory 類)

- This line imports the ChatMessageHistory class from the langchain.memory module. This class is used to handle the chat messages in memory.
- 這行代碼從 langchain.memory 模塊中導入 ChatMessageHistory 類。此類用於在內存中處理聊天消息。

In [ ]:
from langchain.memory import ChatMessageHistory

### 2. Creating an Instance of ChatMessageHistory (創建 ChatMessageHistory 的實例)

- This line creates an instance of ChatMessageHistory. This instance will store the chat messages in memory for this session.
- 這行代碼創建一個 ChatMessageHistory 的實例。該實例將在此會話期間將聊天消息存儲在內存中。

In [ ]:
demo_chat_history = ChatMessageHistory()

### 3. Adding User and AI Messages (添加用戶和 AI 消息)

- demo_chat_history.add_user_message("hi!") adds a user message ("hi!") to the chat history.
- demo_chat_history.add_ai_message("whats up?") adds an AI response ("whats up?") to the chat history.
- demo_chat_history.add_user_message("hi!") 將用戶消息（“hi!”）添加到聊天記錄中。
- demo_chat_history.add_ai_message("whats up?") 將 AI 回應（“whats up?”）添加到聊天記錄中。

In [ ]:
demo_chat_history.add_user_message("hi!")

demo_chat_history.add_ai_message("whats up?")

demo_chat_history.messages

In [ ]:
demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("Translate this sentence from English to  Chinese (繁體中文): I love programming.")

demo_chat_history.add_ai_message("我愛程式設計.")

demo_chat_history.messages

### 4. Retrieving the Messages (檢索消息)

- This line retrieves the list of messages stored in demo_chat_history. Each message is an object that contains information about the sender (user or AI) and the content of the message.
- 這行代碼檢索存儲在 demo_chat_history 中的消息列表。每條消息都是一個對象，包含有關發送者（用戶或 AI）和消息內容的信息。

In [ ]:
{"messages": demo_chat_history.messages}

In [ ]:
demo_chat_history.add_user_message(
    "What did you just say?"
)

response = pipeline_.invoke({"messages": demo_chat_history.messages})

response

# **** 預計第一個小時結束 ****

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from src.initialization import credential_init


credential_init()

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-2024-05-13", temperature=0)

## Conversational Retrievers - Step 1

- 土味情話反殺大全 (推薦上Youtube看)

In [ ]:
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

df = pd.DataFrame(data=[["确认过眼神，你是我爱的人。", "确认过眼神，我是你泡不到的人。"],
                         ["万水千山总是情，爱我多一点行不行。", "一寸光阴一寸金，劝你死了这条心。"],
                         ["今天吃了泡面，吃了炒面，还是想走进你的心里面。", "吃那么多面，最后还不是变成大便。"],
                         ["草莓，蓝莓，蔓越莓，今天你想我了没？", "冬瓜，西瓜，哈密瓜，你再巴巴我打得你叫妈妈。"],
                         ["众生皆苦，唯你独甜。", "尝遍众生，你为渣男代言。"],
                         ["你喜欢瑞士名表还是我帅气的外表？", "我喜欢去年买了个表。"],
                         ["我想问一条路，到哥哥心里的路。", "山路十八弯，走完脑血栓。"],
                         ["小姐姐，我心里给你留了一块地，死心塌地。", "对不起，我的心里只容得下一块地，玛莎拉蒂。"],
                         ["小姐姐你笑起来真好看啊。", "你看起来真好笑啊。"],
                         ["亲爱的你知道吗，你的笑容没有酒，我却醉得像条狗", "我的笑容没有酒，你是真的像条狗"],
                         ["宝贝儿，我在手上划了一道口子，你也划一下吧，这样我们就是两口子了", "我怕我们的血溶到一起，被你发现其实我是你爸爸"],
                         ["这世间万物都有尽头，落叶归根，而我归你", "对不起 我不收垃圾"],
                         ["请问……我想问一下路，那条通往你心里的路", "八格牙路"],
                         ["你今天怎么怪怪的？ 怪可爱的",  "你今天也怪怪的，怪恶心的"],
                         ["亲爱的，你知道我和唐僧的区别吗？ 唐僧取经我娶你", "知道你和沙僧的区别吗？ 他叫沙僧你叫沙雕"],
                         ["亲爱的，你不觉得累吗？ 你已经在我的脑海里跑了好几圈了", "傻孩子，我在找出口呢"],
                         ["莫文蔚的阴天。孙燕姿的雨天，周杰伦的晴天，都不如你和我聊天", "求求你了，能否还我一个宁静的夏天"],
                         ["如果你是方便面，那我就是白开水，今生今世，我泡定你了", "故事的最后，她变成了屎，你变成了尿，你们终究分道扬镳"],
                         ["大年三十晚上的鞭炮再响，也没有我想你那么想", "大年三十晚上的鞭炮再响，也没有你放的屁响"],
                         ["c罗可以上演帽子戏法，可我想你却没有办法", "c罗可以上演帽子戏法，我也可以给你上演绿帽子戏法"],
                         ["不要抱怨，抱我", "抱不起来，太重"],
                         ["你有没有发现我的眼睛很好看？因为我满眼都是你啊", "对不起，你眼睛在哪呢？"]], 
                  columns=['input', 'output'])

documents = []

for _, row in df.iterrows():
    documents.append(Document(page_content=row['input'], metadata={'output': row['output']}))

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 10})

## Build Chat Chain - Step 2

In [ ]:
from operator import itemgetter

from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate
ChatPromptTemplate, SystemMessagePromptTemplate,  MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain, RunnablePassthrough


def build_standard_chat_prompt_template_v3(kwargs):

    messages = []
    
    for key in ['system', 'messages', 'human']:
        if kwargs.get(key):
            if key == 'system':
                system_content = kwargs['system']
                system_prompt = PromptTemplate(**system_content)
                message = SystemMessagePromptTemplate(prompt=system_prompt)
            elif key == 'human':
                human_content = kwargs['human']
                human_prompt = PromptTemplate(**human_content)
                message = HumanMessagePromptTemplate(prompt=human_prompt)
            else:
                message = MessagesPlaceholder(variable_name="messages")
            
            messages.append(message)
    
    chat_prompt = ChatPromptTemplate.from_messages(messages)

    return chat_prompt



@chain
def chatbot_prompt_fn(data):

    system_template = """
                      You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                      You will respond with the following style, cheesy pickup lines, 
                      shown in the context:\n\n{context}\n
                      You will reply in simplified Chinese (簡體中文).
                      """
    
    human_template = data['input']
    
    input_ = {"system": {"template": system_template},
              "human": {"template": human_template},
              "messages": None}
    
    prompt_template = build_standard_chat_prompt_template_v3(input_)

    return prompt_template

@chain
def context_parser(documents):

    context = ""

    for idx, document in enumerate(documents):
        context += f"Example {idx}:\nQuestion: {document.page_content}\nAnswer: {document.metadata['output']}\n"
    
    return context
    
step_1 = RunnablePassthrough.assign(context=itemgetter("input") | retriever | context_parser)

In [ ]:
demo_chat_history = ChatMessageHistory()

step_1.invoke({"input": "你有个超能力 我也有个超能力",
               "messages": demo_chat_history.messages})

In [ ]:
pipeline_ = step_1 | chatbot_prompt_fn
pipeline_.invoke({"input": "你有个超能力 我也有个超能力",
                  "messages": demo_chat_history.messages})

In [ ]:
pipeline_ = step_1 | chatbot_prompt_fn | model | StrOutputParser()
pipeline_.invoke({"input": "你有个超能力 我也有个超能力",
                  "messages": demo_chat_history.messages})

## Test - Step 3

https://www.wenan.wang/qibaitiaotuweiqinghua.html

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory


chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    answer = pipeline_.invoke({"input": question,
                               "message": chat_history.messages
                              })

    print(answer)
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)

In [ ]:
chat_history

### 回家作業 2: 將retriever抽換成WikipediaRetriever

基本上，你可以將這個retriever的內容抽換成任何你需要的資料，來加快寫報告的效率。記得Double Check....

## Compress the chat history to reduce the size of the prompt


https://github.com/langchain-ai/langserve/blob/main/examples/conversational_retrieval_chain/server.py

### Condensation

In [ ]:
system_template = """
                  Combine the following conversation and a follow up ***USER QUERY***, to generate 
                  a standalone query, in its original language.
                  """

human_template = """
                 ***USER QUERY***: {input}
                 """
input_ = {"system": {"template": system_template},
          "human": {"template": human_template},
          "messages": None}
    
condensed_prompt_template = build_standard_chat_prompt_template_v3(input_)

condensed_pipeline = condensed_prompt_template | model | StrOutputParser()

# condensed_chain = {"question": itemgetter("question"),
#                    "messages": itemgetter("message")} | condensed_chat_template | model | StrOutputParser()

## Retrieval

How to implement this properly?

Let start from a higher point of view

In [ ]:
from langchain_core.runnables import RunnablePassthrough

system_template = """
                  You are a helpful AI assistant acting as if you hava rough day and you are now very grumpy. 
                  You will respond with the following style, cheesy pickup lines, 
                  shown in the context:\n\n{context}\n
                  You will reply in simplified Chinese (簡體中文).
                  """
    
human_template = """
                 {standalone_question}
                 """

input_ = {"system": {"template": system_template,
                     "input_variable": ["context"]},
          "human": {"template": human_template,
                    "input_variable": ["standalone_question"]}
         }

chat_prompt_template = build_standard_chat_prompt_template_v2(input_)

step_1 = RunnablePassthrough.assign(context=itemgetter("input") | retriever | context_parser)

retrieval_pipeline = step_1|RunnablePassthrough.assign(answer=chat_prompt_template|model|StrOutputParser())

In [ ]:
final_pipeline = RunnablePassthrough.assign(standalone_question=condensed_pipeline)|retrieval_pipeline

In [ ]:
chat_history = ChatMessageHistory()

final_pipeline.invoke({"messages": chat_history.messages,
                       "input": "你有个超能力 我也有个超能力"})

In [ ]:
chat_history = ChatMessageHistory()

while True:
    question = input("Please input your question: ")

    # if you want to quit
    if question == "QUIT":
        break
    
    output = final_pipeline.invoke({"input": question,
                                    "message": chat_history.messages
                                   })

    print(output)
    
    answer = output['answer']
    
    chat_history.add_user_message(question)
    chat_history.add_ai_message(answer)

## OpenAI Model Fine-tuning

剛好有人來找我家教這個，整理了一下。。。

參考:

- https://cookbook.openai.com/examples/chat_finetuning_data_prep

- {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]} {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]} {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}

In [ ]:
import json

import pandas as pd
from sklearn.model_selection import train_test_split

from src.io.path_definition import get_project_dir

jsonl = []

with open(os.path.join(get_project_dir(), 'tutorial', 'LLM+Langchain', 'Week-1', 'recipe_train.json'), 'r') as f:
    recipe_train = json.load(f)

### Create a jsonl format

In [ ]:
recipe_df = pd.DataFrame(data=[[rec['id'], rec['cuisine'], ", ".join(rec['ingredients'])] 
                               for rec in recipe_train], columns=['id', 'cuisine', 'ingredients'])

"""
train-test split

- 固定random state，確保數據的重現性
- 使用分層抽樣(stratified sampling)，保證訓練-測試集的class分佈是一致的
"""

train, test = train_test_split(recipe_df, test_size=0.2, random_state=42, stratify=recipe_df['cuisine'].tolist())

In [ ]:
system_prompt_template = """You are a helpful AI assistant as a chef of a Michellin 3 stars restaurant. You have extensive knowledge about cuisines 
                            all over the world, and you are able to identify the origin of a cuisine based on the ingredients. You are assigned with a 
                            task of identifying the origin, as region, of cuisine based on the <ingredients>.                            
                            """

for _, row in train.iterrows():

    ingredients = row['ingredients']
    content = f"ingredients: [{ingredients}]"
    
    jsonl.append({"messages": [{"role": "system", "content": system_prompt_template}, 
                               {"role": "user", "content": content}, 
                               {"role": "assistant", "content": row['cuisine']}]})

In [ ]:
# 寫入檔案

with open('openapi_finetuning_test.jsonl', 'w') as outfile:
    for entry in jsonl:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
# 直接使用OpenAI提供的API

from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

client.files.create?

In [ ]:
client.files.create(file=open('openapi_finetuning_test.jsonl', 'rb'),
                    purpose='fine-tune')

In [ ]:
client.fine_tuning.jobs.create?

In [ ]:
client.fine_tuning.jobs.create(model="gpt-4o-mini-2024-07-18",
                               training_file="file-SHpXusOxxMVGYRgwIGNj6mk9",
                               hyperparameters={"batch_size":4, "learning_rate_multiplier": 1e-6, "n_epochs": 5},
                               suffix='test')

訓練完之後

In [ ]:
model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="ft:gpt-4o-mini-2024-07-18:cosnova-account:test:ANPq9Weh", 
                   temperature=0 # a range from 0-2, the higher the value, the higher the `creativity`
                  )

In [ ]:
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate

system_prompt = PromptTemplate.from_template(system_prompt_template)

system_message = SystemMessagePromptTemplate(prompt=system_prompt)

human_prompt = PromptTemplate(template="""
                                       ingredients: [{ingredients}]
                                       """,
                              input_variables=["ingredients"]
                              )

human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message,
                                                human_message
                                                ])

fine_tuned_chain = chat_prompt|model


In [ ]:
fine_tuned_chain.invoke({"ingredients": test.iloc[0]['ingredients']})

In [ ]:
model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini-2024-07-18", 
                   temperature=0 # a range from 0-2, the higher the value, the higher the `creativity`
                  )

chain = chat_prompt|model

In [ ]:
chain.invoke({"ingredients": test.iloc[0]['ingredients']})